In [53]:
import os
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Carregando a base de dados

In [54]:
data = pd.read_csv('loan.csv')

In [55]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               601 non-null object
Married              611 non-null object
Dependents           599 non-null object
Education            614 non-null object
Self_Employed        582 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB


In [57]:
data.Loan_Status.value_counts()

Y    422
N    192
Name: Loan_Status, dtype: int64

In [58]:
data2 = data[data.Loan_Status=='Y'].sample(200)

In [59]:
data = data2.append(data[data.Loan_Status=='N'].sample(192))

In [60]:
data.Loan_Status.value_counts()

Y    200
N    192
Name: Loan_Status, dtype: int64

## Checando Missing Values

In [61]:
data2.isnull().sum()

Loan_ID               0
Gender                1
Married               0
Dependents            4
Education             0
Self_Employed        12
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      4
Credit_History       18
Property_Area         0
Loan_Status           0
dtype: int64

Preechendo Missing Values:

- `Dependents`: Assumindo o valor majoritário da coluna.
- `Self_Employed`: Assumindo o valor majoritário da coluna.
- `Loan_Amount_Term`: Preenchendo com o valor médio da coluna.
- `Credit_History`: Assumindo o valor majoritário da coluna.
- `Married`: Assumindo o valor majoritário da coluna.
- `Gender`: Assumindo o valor majoritário da coluna.

In [62]:
data['Gender'] = data['Gender'].fillna('Male')

In [63]:
data['Married'] = data['Married'].fillna('No')

In [64]:
data['Dependents'] = data['Dependents'].fillna('0')

In [65]:
data['Self_Employed'] = data['Self_Employed'].fillna('No')

In [66]:
data['LoanAmount'] = data['LoanAmount'].fillna(data['LoanAmount'].mean())

In [67]:
data['Credit_History'] = data['Credit_History'].fillna(1.0)

In [68]:
data['Loan_Amount_Term'] = data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mean())

In [69]:
data.Credit_History.value_counts()

1.0    306
0.0     86
Name: Credit_History, dtype: int64

### Checando novamente Missing Values

In [70]:
data.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

**Transformando dados categóricos**

Várias colunas do dataframe são categóricas, precisamos transforma-las, são elas: `Gender`, `Married`, `Education`, `Self_Employed` & `Property_Area` columns.

In [71]:
from sklearn.preprocessing import LabelEncoder

In [72]:
gender_values = {'Female' : 0, 'Male' : 1} 
married_values = {'No' : 0, 'Yes' : 1}
education_values = {'Graduate' : 0, 'Not Graduate' : 1}
employed_values = {'No' : 0, 'Yes' : 1}
dependent_values = {'3+': 3, '0': 0, '2': 2, '1': 1}
loan_values = {'Y':1,'N':0}
data.replace({'Gender': gender_values,
                 'Married': married_values, 
                 'Education': education_values,
                 'Self_Employed': employed_values, 
                 'Dependents': dependent_values,
                 'Loan_Status': loan_values
                }, inplace=True)

In [73]:
data.drop(['Loan_ID','CoapplicantIncome','Loan_Amount_Term','Credit_History','Property_Area'],axis=1,inplace=True)

In [74]:
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Status
454,1,0,0,0,1,7085,84.0,1
55,1,1,2,0,0,2708,97.0,1
436,1,0,0,0,0,1926,50.0,1
490,1,0,0,1,0,2699,96.0,1
601,1,1,0,1,0,2894,155.0,1


Selecionando o melhor classificador através de Pipeline e GridSearchCV

In [75]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

In [76]:
pipe_random_forest = Pipeline([
        ('scl', StandardScaler()),
        
        ('clf', RandomForestClassifier())
        ]
)

In [77]:
pipe_svm = Pipeline([
        ('scl', StandardScaler()),
        ('clf', svm.SVC())
        ]
)

In [78]:
pipe_knn = Pipeline([
        ('scl', StandardScaler()),
        ('clf', KNeighborsClassifier())
        ]
)

#### Valores para Grid

In [79]:
valores = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [80]:
grid_params_rf = [{
    'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_leaf': valores,
    'clf__max_depth': valores,
    'clf__min_samples_split': valores[1:]
}]

In [81]:
grid_params_svm = [{
    'clf__kernel': ['linear', 'rbf'], 
    'clf__C': valores
}]

In [82]:
grid_params_knn = [{
    'clf__n_neighbors': valores,
}]

#### Construindo GridSearch

In [83]:
gs_rf = GridSearchCV(
    estimator=pipe_random_forest,
    param_grid=grid_params_rf,
    scoring='accuracy',
    cv=10 
)

In [84]:
gs_svm = GridSearchCV(
    estimator=pipe_svm,
    param_grid=grid_params_svm,
    scoring='accuracy',
    cv=10,
)

In [85]:
gs_knn = GridSearchCV(
    estimator=pipe_knn,
    param_grid=grid_params_knn,
    scoring='accuracy',
    cv=10,
)

In [86]:
X_train = data.drop('Loan_Status',axis=1)
y = data['Loan_Status']

#### Computando o GridSearch para Random Forest

In [87]:
gs_rf.fit(X_train,y)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
        ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'clf__criterion': ['gini', 'entropy'], 'clf__min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'clf__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'clf__min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

#### Melhores parametros e scoring

In [88]:
print('Melhores parâmetros: %s' % gs_rf.best_params_)
print('Melhores Acurácia: %.3f' % gs_rf.best_score_)

Melhores parâmetros: {'clf__criterion': 'gini', 'clf__max_depth': 3, 'clf__min_samples_leaf': 3, 'clf__min_samples_split': 8}
Melhores Acurácia: 0.579


#### Computando o GridSearch para SVM

In [ ]:
gs_svm.fit(X_train,y)

#### Melhores parametros e scoring

In [ ]:
print('Melhores parâmetros: %s' % gs_svm.best_params_)
print('Melhores Acurácia: %.3f' % gs_svm.best_score_)

#### Computando o GridSearch para KNN

In [ ]:
gs_knn.fit(X_train,y)

#### Melhores parametros e scoring

In [ ]:
print('Melhores parâmetros: %s' % gs_knn.best_params_)
print('Melhores Acurácia: %.3f' % gs_knn.best_score_)

### Métricas de Validação ###

In [89]:
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [90]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X_train, y)

In [91]:
print (pd.crosstab(y_teste, gs_rf.predict(X_teste), rownames=['Real'], colnames=['Predito'], margins=True), '')

Predito   0   1  All
Real                
0        21  25   46
1        10  42   52
All      31  67   98 


In [92]:
print (metrics.classification_report(y_teste,gs_rf.predict(X_teste)))

              precision    recall  f1-score   support

           0       0.68      0.46      0.55        46
           1       0.63      0.81      0.71        52

   micro avg       0.64      0.64      0.64        98
   macro avg       0.65      0.63      0.63        98
weighted avg       0.65      0.64      0.63        98



### Persistindo o modelo de Machine Learning para o disco. ###

In [93]:
from sklearn.externals import joblib

#### Persistindo o melhor modelo em disco.

In [94]:
joblib.dump(gs_rf, 'model.pkl')

['model.pkl']

#### Listando os arquivos em disco.

In [95]:
!ls

'ls' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


#### Carregando o modelo a partir do disco para a memória.

In [96]:
model = joblib.load('model.pkl')

In [97]:
print("Atributos do Modelo:\n\nClasses:{}\nEstimator:{}".format(model.classes_,model.estimator))

Atributos do Modelo:

Classes:[0 1]
Estimator:Pipeline(memory=None,
     steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
        ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])


**Verificando o Dataset final gerado.**

In [ ]:
X_train.head()

**Teste de Classificação.**

In [ ]:
teste = np.array([[1,1,3,0,0,9504,275.0]])

In [ ]:
model.predict(teste)

**Probabilidades de Classes.**

In [ ]:
model.predict_proba(teste)